In [22]:
import requests
import json

query = {}

# queries must be formatted in Lucene ElasticSearch style
# https://www.elastic.co/guide/en/elasticsearch/reference/current/full-text-queries.html

# here are different query types :

# MULTI_MATCH queries
# use it for searching in specific fields, with a general OR or AND

search_fields = ["title","abstract","keywords","mesh_terms"]
# you can use "title^3" for boosting by 3 scores in title
keywords = "BRCA2 cancer"
my_operator = "and" # default is "or"
my_type = "phrase" # use it for phrase matching (exact expression)

my_query = {
    "size": 10, # maximum amount of hits returned
    "from" : 0, # offset from the first result you want to fetch
    "query": {
        "multi_match" : {
            "query" : keywords,
            "fields" : search_fields
            #,"operator" : my_operator
            #,"type" = "phrase"
            }
        }
    }

In [23]:
# ex: ("EML4" OR "ALK" in title) AND ("Lung Cancer" in journal)
my_query = {
    "query": {
        "bool" : {
            "must" :[ # AND
                {
                    "bool" : {
                        "should" :[ # OR
                            {
                                "match" : {"title": "EML4"},
                                "match" : {"title": "ALK"}
                                }
                            ]
                        }
                    },
                {
                    "match" : {"journal": "Lung Cancer"}
                    }
                ]
            }
        }
    }

In [37]:
# call
url_API = "http://candy.hesge.ch/SIBiLS/MEDLINE/search.jsp"
my_json_query = json.dumps(my_query) # json to string
my_params = {"json_query": my_json_query} # parameters dictionary
r = requests.post(url = url_API, params = my_params)

# get response and print in output file
response = r.text
with open("SIBiLS_MED_search.json","w",encoding="utf-8") as file:
   file.write(r.text)

parsed = json.loads(response)
print(json.dumps(parsed['hits'], indent = 2))

[
  {
    "_index": "med20_v2",
    "_type": "citation",
    "_id": "32576444",
    "_score": 24.198877,
    "_source": {
      "title": "Chr2 30297612-ALK, A Novel Intergenic Fusion With Exon18 of ALK, Responds to Crizotinib.",
      "abstract": "",
      "journal": "Clinical lung cancer",
      "authors": "Chen X|Zhao G|Zhong P|Zhang M|Chen R|Zhang D",
      "affiliations": "Department of Thoracic Surgery, Fujian Province Cancer Hospital, Fuzhou, Fujian, China.|Department of Thoracic Surgery, Fujian Province Cancer Hospital, Fuzhou, Fujian, China.|Department of Gynecological Oncology Radiotherapy, Fujian Province Cancer Hospital, Fuzhou, Fujian, China.|Department of Medical Center, Geneplus-Beijing Ltd., Beijing, China.|Department of Medical Center, Geneplus-Beijing Ltd., Beijing, China.|Department of Thoracic Surgery, Fujian Province Cancer Hospital, Fuzhou, Fujian, China. Electronic address: surgzhangderong@163.com.",
      "pubyear": 2020,
      "entrez_date": "2020-06-25",
      

In [51]:
for list_item in parsed['hits']:
    source_item = list_item['_source']
    print(source_item['pmid'],source_item['title'],source_item['title'])

32576444 Chr2 30297612-ALK, A Novel Intergenic Fusion With Exon18 of ALK, Responds to Crizotinib. Chr2 30297612-ALK, A Novel Intergenic Fusion With Exon18 of ALK, Responds to Crizotinib.
27130468 Anaplastic Lymphoma Kinase (ALK) Kinase Domain Mutation Following ALK Inhibitor(s) Failure in Advanced ALK Positive Non-Small-Cell Lung Cancer: Analysis and Literature Review. Anaplastic Lymphoma Kinase (ALK) Kinase Domain Mutation Following ALK Inhibitor(s) Failure in Advanced ALK Positive Non-Small-Cell Lung Cancer: Analysis and Literature Review.
31147208 Relevance of Detection of Mechanisms of Resistance to ALK Inhibitors in ALK-Rearranged NSCLC in Routine Practice. Relevance of Detection of Mechanisms of Resistance to ALK Inhibitors in ALK-Rearranged NSCLC in Routine Practice.
24984564 ALK inhibitors: what is the best way to treat patients with ALK+ non-small-cell lung cancer? ALK inhibitors: what is the best way to treat patients with ALK+ non-small-cell lung cancer?
33277054 GPC1-ALK: A

In [66]:
def genOrgan(gene,organ):
    
    import requests
    import json

    query = {}

    # queries must be formatted in Lucene ElasticSearch style
    # https://www.elastic.co/guide/en/elasticsearch/reference/current/full-text-queries.html

    # here are different query types :

    # MULTI_MATCH queries
    # use it for searching in specific fields, with a general OR or AND

    search_fields = ["title","abstract","keywords","mesh_terms"]
    # you can use "title^3" for boosting by 3 scores in title
    keywords = gene + " " + organ
    my_operator = "and" # default is "or"
    my_type = "phrase" # use it for phrase matching (exact expression)

    my_query = {
        "size": 10, # maximum amount of hits returned
        "from" : 0, # offset from the first result you want to fetch
        "query": {
            "multi_match" : {
                "query" : keywords,
                "fields" : search_fields
                #,"operator" : my_operator
                #,"type" = "phrase"
                }
            }
    }
    
    # query for required gene and organ
    
    my_query = {
    "query": {
        "bool" : {
            "must" :[ # AND
                {
                    "bool" : {
                        "should" :[ # OR
                            {
                                "match" : {"title": gene},
                                "match" : {"title": organ}
                                }
                            ]
                        }
                    },
                {
                    "bool" : {
                        "should" :[ # OR
                            {
                                "match" : {"abstract": gene},
                                "match" : {"abstract": organ}
                                }
                            ]
                        }
                    },
                ]
            }
        }
    }
    
    # call
    url_API = "http://candy.hesge.ch/SIBiLS/MEDLINE/search.jsp"
    my_json_query = json.dumps(my_query) # json to string
    my_params = {"json_query": my_json_query} # parameters dictionary
    r = requests.post(url = url_API, params = my_params)

    # get response and print in output file
    response = r.text
    with open("SIBiLS_MED_search.json","w",encoding="utf-8") as file:
       file.write(r.text)

    parsed = json.loads(response)
    
    for list_item in parsed['hits']:
        source_item = list_item['_source']
        print(source_item['pmid']+'\n'+source_item['title']+'\n'+source_item['abstract'])
        print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')

In [67]:
genOrgan('alk','lung')

10694605
Heart-lung versus double-lung transplantation for suppurative lung disease.
OBJECTIVE
The purpose of this study was to compare outcomes after heart-lung or double-lung transplantation in patients undergoing transplantation because of end-stage suppurative lung disease.


METHODS
We reviewed our experience in patients with cystic fibrosis or bronchiectasis who had heart-lung or double-lung transplantation between January 1988 and September 1997. Twenty-three patients (14 male, 21 cystic fibrosis) had heart-lung transplantation and 24 patients (8 male, 19 cystic fibrosis) had double-lung transplantation. There were no statistically significant differences between the groups in age, weight, preoperative creatinine level, cytomegalovirus status, maintenance immunosuppression, or donor demographics. Patients received induction therapy with monoclonal (OKT3) or polyclonal (rabbit anti-thymocyte globulin) antibody.


RESULTS
Sixteen of 24 patients had double-lung transplantation afte